In [1]:
from openai import OpenAI
client = OpenAI()

In [3]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  
  def __init__(self):
    self.text_delta_values = ''
    self.image_id = ''
    super().__init__()
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          print(output.type)
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
          elif output.type == "image":
            print(type(output.image))
            image_id = output.image.file_id
            print(image_id)
            self.image_id = image_id
            image_data = client.files.content(image_id)
            image_data_bytes = image_data.read()

            with open(f"./image-{image_id}.png", "wb") as file:
                file.write(image_data_bytes)

In [8]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What day of the week were the most bookings made? Generate code to answer this and display a visualization."
)

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id="asst_EKZ2miOCbP7AayhZLCSzhkQK", # asst_EKZ2miOCbP7AayhZLCSzhkQK OR  asst_gwH0YPXzLZpaUoNnGQzmkOaG (3)
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > To start, I need to inspect the contents of the files you've uploaded to understand their structure and the relevant data they contain, such as booking dates, which I'll use to answer your question. Let's begin by loading the data from the files and examining the first few rows.
assistant > code_interpreter

import pandas as pd

# Load the data from the uploaded files
file_path1 = "/mnt/data/file-4FKpqlvJOZqxNUfIJHAHRGJY"
file_path2 = "/mnt/data/file-TzOVAHko5TwyrRsaPySmIR0Z"

# Trying to read the files to see their format and content
data1 = pd.read_csv(file_path1)
data2 = pd.read_csv(file_path2)

data1.head(), data2.head()

output >
logs

(                               id_guest                               id_host  \
 0  000dfad9-459b-4f0b-8310-3d6ab34e4f57  13bb24b8-d432-43a2-9755-5ea11b43bb69   
 1  00197051-c6cb-4c3a-99e9-86615b819874  46aa3897-9c00-4d76-ac66-a307593d0675   
 2  0027538e-aa9e-4a02-8979-b8397e5d4cba  6bbb88ca-db66-48c5-9c4b-862f7706284a   
 3  002753